In [0]:
from sparkcuml.decomposition import SparkCumlPCA
import numpy as np
import pandas as pd
import time


### Create synthetic dataset

In [0]:
num_vecs = 100000
dim = 3000
vec = np.random.randn(dim).reshape([1,dim])
arr = np.random.randn(num_vecs).reshape([num_vecs,1])
data = arr * vec
data = data

In [0]:
pd_data = pd.DataFrame({"features": list(data)})

In [0]:
df = spark.createDataFrame(pd_data)

### Run Cuml-on-Spark PCA.fit

In [0]:
num_workers = 2
topk = 2

In [0]:
gpu_pca = SparkCumlPCA(num_workers=num_workers,verbose=6).setInputCol("features").setK(topk)

PCA estimator can be persisted and reloaded.

In [0]:
gpu_pca.write().overwrite().save("/tmp/sparkcuml-pca-estimator")
gpu_pca_loaded = SparkCumlPCA.load("/tmp/sparkcuml-pca-estimator")


In [0]:
start_time = time.time()
gpu_model = gpu_pca_loaded.fit(df)
print(f"Fit took: {time.time() - start_time} sec")

Fit took: 37.46277117729187 sec

In [0]:
gpu_model.explained_variance

Out[9]: [2893.218342209522, 5.457716536556868e-14]

In [0]:
gpu_model.mean[0:10]

Out[10]: [-0.0015620115605000174,
 -0.002377551657468376,
 0.0032997137789986773,
 0.0007433972190390371,
 -0.0012920472737014475,
 0.0010460212210790844,
 0.0015153518941659091,
 0.00014012827761658553,
 0.0013196662484463784,
 0.0018075545735492253]

In [0]:
[component[0:10] for component in gpu_model.pc]

Out[11]: [[-0.01952620814340519,
 -0.029721014689906896,
 0.041248669146702895,
 0.009292971447367626,
 -0.016151470728768214,
 0.013075977541850602,
 0.0189429305417658,
 0.0017516988892466742,
 0.01649672671996229,
 0.02259566300673727],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]

In [0]:
np.divide(vec,gpu_model.pc[0])[0][0:10]

Out[12]: array([53.87583902, 53.87583902, 53.87583902, 53.87583902, 53.87583902,
 53.87583902, 53.87583902, 53.87583902, 53.87583902, 53.87583902])

### Cuml-on-Spark PCA model save, load, and transform

In [0]:
gpu_model.write().overwrite().save("dbfs:/tmp/sparkcuml-pca-model")

In [0]:
gpu_model_loaded = gpu_model.load("dbfs:/tmp/sparkcuml-pca-model")

In [0]:
gpu_model_loaded.mean[0:10]

Out[15]: [-0.0015620115605000174,
 -0.002377551657468376,
 0.0032997137789986773,
 0.0007433972190390371,
 -0.0012920472737014475,
 0.0010460212210790844,
 0.0015153518941659091,
 0.00014012827761658553,
 0.0013196662484463784,
 0.0018075545735492253]

In [0]:
gpu_model_loaded.explained_variance

Out[16]: [2893.218342209522, 5.457716536556868e-14]

In [0]:
[component[0:10] for component in gpu_model_loaded.pc]

Out[17]: [[-0.01952620814340519,
 -0.029721014689906896,
 0.041248669146702895,
 0.009292971447367626,
 -0.016151470728768214,
 0.013075977541850602,
 0.0189429305417658,
 0.0017516988892466742,
 0.01649672671996229,
 0.02259566300673727],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]

In [0]:
transformed_df = gpu_model.setOutputCol("transformed").transform(df)

In [0]:
transformed_df.printSchema()

root
-- transformed: array (nullable = false)
 |-- element: double (containsNull = false)

In [0]:
transformed_df.count()

Out[20]: 100000

In [0]:
transformed_df.show(10)

+--------------------+
 transformed|
+--------------------+
[49.4851772638877...|
[-23.273387957923...|
[-84.966439607713...|
[62.9931993953471...|
[32.7787391090148...|
[44.1321218542269...|
[30.3136428042469...|
[112.187064920278...|
[-56.612936849433...|
[-101.41778953587...|
+--------------------+
only showing top 10 rows

### Run Spark ML PCA.fit

In [0]:
from pyspark.ml.feature import PCA

In [0]:
spark_ml_pca = PCA().setInputCol("features").setK(topk)

Convert array sql type to VectorUDT expected by Spark ML algos

In [0]:
from pyspark.ml.functions import array_to_vector

In [0]:
vector_df = df.select(array_to_vector(df.features).alias("features"))

In [0]:
vector_df.printSchema()

root
-- features: vector (nullable = true)

In [0]:
start_time = time.time()
spark_ml_pca_model = spark_ml_pca.fit(vector_df)
print(f"Fit took: {time.time() - start_time} sec")

Fit took: 276.28289675712585 sec

In [0]:
spark_ml_pca_model.explainedVariance

Out[28]: DenseVector([1.0, 0.0])

In [0]:
pc = spark_ml_pca_model.pc

In [0]:
np.divide(vec,pc.toArray()[:,0])[0][0:10]

Out[30]: array([53.87583902, 53.87583902, 53.87583902, 53.87583902, 53.87583902,
 53.87583902, 53.87583902, 53.87583902, 53.87583902, 53.87583902])

### Running cuml.PCA.fit single-node on driver

In [0]:
from cuml import PCA

In [0]:
cuml_pca = PCA(n_components=topk)

In [0]:
cuml_model = cuml_pca.fit(data[:100000,:])

In [0]:
cuml_model.explained_variance_

Out[34]: array([2.89321834e+03, 1.68261720e-12])

In [0]:
cuml_model.explained_variance_ratio_

Out[35]: array([1.00000000e+00, 5.81572838e-16])